In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import copy
import json
import geopandas as gpd
from arcgis.gis import GIS

In [2]:
from arcgis.geoenrichment import enrich

In [3]:
# init GIS connection
with open("agol_account_info.txt", "r") as f:
    url, username, password = f.read().splitlines()

In [4]:
gis = GIS(url, username=username, password=password)

In [9]:
sd_bgs_enriched = gis.content.get('7b2d2122d70c4d61ad86d92f77ab4fc4')

In [14]:
fset = sd_bgs_enriched.layers[0].query()

In [16]:
gjson_string = fset.to_geojson

In [17]:
with open('sd_bgs_enriched.json', 'w') as f:
    f.write(gjson_string)

In [19]:
sd_bgs_enriched_sdf = pd.DataFrame.spatial.from_layer(sd_bgs_enriched.layers[0])

In [22]:
list(sd_bgs_enriched_sdf.columns)

['AGE_10_14',
 'AGE_15_19',
 'AGE_20_24',
 'AGE_25_34',
 'AGE_35_44',
 'AGE_45_54',
 'AGE_55_64',
 'AGE_5_9',
 'AGE_65_74',
 'AGE_75_84',
 'AGE_85_UP',
 'AGE_UNDER5',
 'AMERI_ES',
 'ASIAN',
 'AVE_FAM_SZ',
 'AVE_HH_SZ',
 'BLACK',
 'BLKGRP',
 'CNTY_FIPS',
 'ENRICH_FID',
 'FAMILIES',
 'FEMALES',
 'FHH_CHILD',
 'FIPS',
 'HAWN_PI',
 'HISPANIC',
 'HOUSEHOLDS',
 'HSEHLD_1_F',
 'HSEHLD_1_M',
 'HSE_UNITS',
 'HasData',
 'ID',
 'MALES',
 'MARHH_CHD',
 'MARHH_NO_C',
 'MED_AGE',
 'MED_AGE_F',
 'MED_AGE_M',
 'MHH_CHILD',
 'MULT_RACE',
 'N01_BUS',
 'N02_BUS',
 'N03_BUS',
 'N04_BUS',
 'N05_BUS',
 'N06_BUS',
 'N07_BUS',
 'N08_BUS',
 'N09_BUS',
 'N10_BUS',
 'N11_BUS',
 'N12_BUS',
 'N13_BUS',
 'N14_BUS',
 'N15_BUS',
 'N16_BUS',
 'N17_BUS',
 'N18_BUS',
 'N21_BUS',
 'N22_BUS',
 'N23_BUS',
 'N24_BUS',
 'N25_BUS',
 'N26_BUS',
 'N27_BUS',
 'N28_BUS',
 'N29_BUS',
 'N30_BUS',
 'N31_BUS',
 'N32_BUS',
 'N33_BUS',
 'N34_BUS',
 'N35_BUS',
 'N36_BUS',
 'N37_BUS',
 'N38_BUS',
 'N39_BUS',
 'N40_BUS',
 'N41_BUS',
 'OBJ